In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

2023-11-14 16:50:27.468472: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-14 16:50:27.516610: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
fashion_mnist = keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

X_valid, X_train = X_train_full[:5000] / 255., X_train_full[5000:] / 255.
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
X_test = X_test / 255.

In [3]:
class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
               "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

In [4]:
def build_model (n_hidden=3, n_units=100, learning_rate=0.01):
    model = keras.models.Sequential()
    model.add(keras.layers.Flatten(input_shape=[28, 28]))
    for i in range(n_hidden):
        model.add(keras.layers.Dense(n_units, activation="relu"))
    model.add(keras.layers.Dense(10, activation="softmax"))

    model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(lr=learning_rate),
              metrics=["accuracy"])
    return model

In [5]:
from scipy.stats import reciprocal
import sklearn
from sklearn.model_selection import RandomizedSearchCV

param_distribs = {
    "n_hidden": [0, 1, 2, 3],
    "n_units": np.arange(1, 100).tolist(),
    "learning_rate": reciprocal(3e-4, 3e-2).rvs(1000).tolist(),
}

# wrapper
from scikeras.wrappers import KerasClassifier

keras_clf = KerasClassifier(
    model=build_model,
    loss="sparse_categorical_crossentropy",
    n_units=1, n_hidden=1, learning_rate=0.001 # necessary to assign a value here, to make them visible (and tunable) to scikit-learn
)

rnd_search_cv = RandomizedSearchCV(keras_clf, param_distribs, n_iter=2, cv=2, verbose=2) # toy params
rnd_search_cv.fit(X_train, y_train, epochs=45,
                  validation_data=(X_valid, y_valid),
                  callbacks=[keras.callbacks.EarlyStopping(patience=5)])

2023-11-14 16:50:29.281418: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Fitting 2 folds for each of 2 candidates, totalling 4 fits
Epoch 1/45
860/860 [==============================] - 1s 693us/step - loss: 1.3602 - accuracy: 0.5557 - val_loss: 0.9353 - val_accuracy: 0.6628
Epoch 2/45
860/860 [==============================] - 0s 568us/step - loss: 0.8583 - accuracy: 0.6808 - val_loss: 0.7960 - val_accuracy: 0.7036
Epoch 3/45
860/860 [==============================] - 0s 570us/step - loss: 0.7667 - accuracy: 0.7217 - val_loss: 0.7203 - val_accuracy: 0.7446
Epoch 4/45
860/860 [==============================] - 0s 566us/step - loss: 0.7114 - accuracy: 0.7475 - val_loss: 0.6870 - val_accuracy: 0.7632
Epoch 5/45
860/860 [==============================] - 0s 571us/step - loss: 0.6727 - accuracy: 0.7651 - val_loss: 0.6719 - val_accuracy: 0.7570
Epoch 6/45
860/860 [==============================] - 0s 570us/step - loss: 0.6441 - accuracy: 0.7760 - val_loss: 0.6183 - val_accuracy: 0.7938
Epoch 7/45
860/860 [==============================] - 1s 598us/step - loss: 0

Epoch 1/45
860/860 [==============================] - 1s 651us/step - loss: 1.6250 - accuracy: 0.4299 - val_loss: 1.2274 - val_accuracy: 0.5418
Epoch 2/45
860/860 [==============================] - 0s 580us/step - loss: 1.1193 - accuracy: 0.5823 - val_loss: 1.0009 - val_accuracy: 0.6296
Epoch 3/45
860/860 [==============================] - 0s 559us/step - loss: 0.9557 - accuracy: 0.6463 - val_loss: 0.8850 - val_accuracy: 0.6810
Epoch 4/45
860/860 [==============================] - 0s 567us/step - loss: 0.8517 - accuracy: 0.6865 - val_loss: 0.7956 - val_accuracy: 0.7120
Epoch 5/45
860/860 [==============================] - 0s 577us/step - loss: 0.7750 - accuracy: 0.7136 - val_loss: 0.7359 - val_accuracy: 0.7290
Epoch 6/45
860/860 [==============================] - 0s 554us/step - loss: 0.7244 - accuracy: 0.7374 - val_loss: 0.6999 - val_accuracy: 0.7460
Epoch 7/45
860/860 [==============================] - 0s 574us/step - loss: 0.6902 - accuracy: 0.7561 - val_loss: 0.6739 - val_accuracy:

Epoch 1/45
860/860 [==============================] - 1s 779us/step - loss: 0.9341 - accuracy: 0.6982 - val_loss: 0.6242 - val_accuracy: 0.7990
Epoch 2/45
860/860 [==============================] - 1s 705us/step - loss: 0.5870 - accuracy: 0.7995 - val_loss: 0.5364 - val_accuracy: 0.8170
Epoch 3/45
860/860 [==============================] - 1s 713us/step - loss: 0.5241 - accuracy: 0.8189 - val_loss: 0.4981 - val_accuracy: 0.8282
Epoch 4/45
860/860 [==============================] - 1s 710us/step - loss: 0.4892 - accuracy: 0.8277 - val_loss: 0.4728 - val_accuracy: 0.8388
Epoch 5/45
860/860 [==============================] - 1s 740us/step - loss: 0.4665 - accuracy: 0.8356 - val_loss: 0.5174 - val_accuracy: 0.8158
Epoch 6/45
860/860 [==============================] - 1s 717us/step - loss: 0.4463 - accuracy: 0.8417 - val_loss: 0.4414 - val_accuracy: 0.8478
Epoch 7/45
860/860 [==============================] - 1s 721us/step - loss: 0.4325 - accuracy: 0.8484 - val_loss: 0.4647 - val_accuracy:

Epoch 1/45
860/860 [==============================] - 1s 787us/step - loss: 0.9211 - accuracy: 0.7014 - val_loss: 0.6180 - val_accuracy: 0.7930
Epoch 2/45
860/860 [==============================] - 1s 714us/step - loss: 0.5799 - accuracy: 0.7999 - val_loss: 0.6133 - val_accuracy: 0.7804
Epoch 3/45
860/860 [==============================] - 1s 705us/step - loss: 0.5172 - accuracy: 0.8197 - val_loss: 0.5183 - val_accuracy: 0.8272
Epoch 4/45
860/860 [==============================] - 1s 718us/step - loss: 0.4832 - accuracy: 0.8307 - val_loss: 0.5161 - val_accuracy: 0.8186
Epoch 5/45
860/860 [==============================] - 1s 733us/step - loss: 0.4629 - accuracy: 0.8370 - val_loss: 0.4507 - val_accuracy: 0.8456
Epoch 6/45
860/860 [==============================] - 1s 716us/step - loss: 0.4448 - accuracy: 0.8417 - val_loss: 0.4706 - val_accuracy: 0.8340
Epoch 7/45
860/860 [==============================] - 1s 728us/step - loss: 0.4313 - accuracy: 0.8464 - val_loss: 0.4512 - val_accuracy:

Epoch 1/45
1719/1719 [==============================] - 1s 723us/step - loss: 0.7674 - accuracy: 0.7459 - val_loss: 0.5459 - val_accuracy: 0.8154
Epoch 2/45
1719/1719 [==============================] - 1s 701us/step - loss: 0.5030 - accuracy: 0.8231 - val_loss: 0.4501 - val_accuracy: 0.8478
Epoch 3/45
1719/1719 [==============================] - 1s 702us/step - loss: 0.4563 - accuracy: 0.8390 - val_loss: 0.5300 - val_accuracy: 0.8018
Epoch 4/45
1719/1719 [==============================] - 1s 684us/step - loss: 0.4299 - accuracy: 0.8487 - val_loss: 0.4055 - val_accuracy: 0.8588
Epoch 5/45
1719/1719 [==============================] - 1s 693us/step - loss: 0.4118 - accuracy: 0.8551 - val_loss: 0.3910 - val_accuracy: 0.8640
Epoch 6/45
1719/1719 [==============================] - 1s 697us/step - loss: 0.3943 - accuracy: 0.8603 - val_loss: 0.3832 - val_accuracy: 0.8698
Epoch 7/45
1719/1719 [==============================] - 1s 676us/step - loss: 0.3823 - accuracy: 0.8646 - val_loss: 0.3734 -

RandomizedSearchCV(cv=2,
                   estimator=KerasClassifier(learning_rate=0.001, loss='sparse_categorical_crossentropy', model=<function build_model at 0x7ff6e92eb1c0>, n_hidden=1, n_units=1),
                   n_iter=2,
                   param_distributions={'learning_rate': [0.0016834549246003507,
                                                          0.02390836445593179,
                                                          0.008731907739399207,
                                                          0.004725396149933918,
                                                          0.0006154014789262349,
                                                          0.0006153331256530192,...
                                                          0.0011470425674025557,
                                                          0.0050214257366256385,
                                                          0.0005703073595961107,
                                                          0.001151888789941251,
                                                          0.0016212311563941985,
                                                          0.0024505367684280496,
                                                          0.01115509254171962,
                                                          0.0007524347058135697,
                                                          0.003203244812844405,
                                                          0.004591455636549436,
                                                          0.000371554118965828, ...],
                                        'n_hidden': [0, 1, 2, 3],
                                        'n_units': [1, 2, 3, 4, 5, 6, 7, 8, 9,
                                                    10, 11, 12, 13, 14, 15, 16,
                                                    17, 18, 19, 20, 21, 22, 23,
                                                    24, 25, 26, 27, 28, 29, 30, ...]},
                   verbose=2)

In [6]:
rnd_search_cv.best_params_

{'n_units': 94, 'n_hidden': 2, 'learning_rate': 0.005432590230265345}

In [12]:
model = rnd_search_cv.best_estimator_.model_

In [14]:
model.evaluate(X_test, y_test)

313/313 [==============================] - 0s 519us/step - loss: 0.3412 - accuracy: 0.8808


[0.3412206470966339, 0.8808000087738037]